In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression

In [3]:
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')


In [6]:
column_count=24

corr_matrix = train_df.corr()

columns_by_correlation_with_price = train_df.corrwith(train_df.SalePrice).apply(abs).sort_values()
correlated_columns = columns_by_correlation_with_price[::-1][1:column_count+1].keys().to_numpy()

In [7]:
def clean_na(data, drop=False):
    if drop:
        data.dropna(inplace=True)
    else:    
        for col in data[data.isna().any(axis=1)].columns:
            data[col].fillna(value=data[col].mean(), inplace=True)
    
    return data

In [8]:
clean_train_df = clean_na(train_df[correlated_columns].copy())

X = clean_train_df
y = train_df['SalePrice']
    
model = LinearRegression()
model.fit(X, y)

print(f"Train score cols={len(correlated_columns)}: {model.score(X, y):.5f}")
print(f"Columns: {correlated_columns}")



Train score cols=24: 0.80256
Columns: ['OverallQual' 'GrLivArea' 'GarageCars' 'GarageArea' 'TotalBsmtSF'
 '1stFlrSF' 'FullBath' 'TotRmsAbvGrd' 'YearBuilt' 'YearRemodAdd'
 'GarageYrBlt' 'MasVnrArea' 'Fireplaces' 'BsmtFinSF1' 'LotFrontage'
 'WoodDeckSF' '2ndFlrSF' 'OpenPorchSF' 'HalfBath' 'LotArea' 'BsmtFullBath'
 'BsmtUnfSF' 'BedroomAbvGr' 'KitchenAbvGr']


In [9]:
X_test = clean_na(test_df[correlated_columns].copy())

y_test_pred = model.predict(X_test)

output = pd.DataFrame({'Id': test_df['Id'], 'SalePrice': y_test_pred})
output.to_csv(f'../data/SimpleLinearRegression{column_count}.csv', index=False)